In [ ]:
import pandas as pd
import networkx as nx
from scipy.spatial import KDTree
import matplotlib.pyplot as plt
from tqdm import tqdm
from joblib import Parallel, delayed
from folium.plugins import TimestampedGeoJson
from geopy.distance import geodesic
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from pandas import DataFrame
import time
import numpy as np
import folium

In [ ]:
# Reading the gps CSV file into a DataFrame
gps = pd.read_csv('../../filter2_malar_des.csv')
gps3 = pd.read_csv('../../filter3_20m_60min_malar_des.csv')
gps4 = pd.read_csv('../../filter4_20m_60min_immobility_malar_des.csv')

# gps.head()

In [ ]:
def create_plot(data):
    # Ambil baris pertama dari DataFrame
    first_row = data.iloc[0]

    # Ambil nilai latitude dan longitude dari baris pertama
    latitude = first_row['latitude']
    longitude = first_row['longitude']
    
    m = folium.Map(location=[latitude, longitude], zoom_start=25)

    # Add CircleMarkers for each point
    for index, row in data.iterrows():
        folium.CircleMarker(
            location=[row["latitude"], row["longitude"]],
            radius=5,  # Marker size
            color="blue",  # Marker color
            fill=True,
            fill_color="blue",  # Fill color of the marker
            fill_opacity=0.7,  # Opacity of the marker fill
            popup=f"User ID: {row['maid']}<br>Latitude: {row['latitude']}<br>Longitude: {row['longitude']}",
        ).add_to(m)
    
    return m

In [ ]:
def create_pivot(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df_pivot = df['maid'].groupby(df['timestamp'].dt.date).value_counts()
    pivot = df_pivot.unstack().fillna(0).astype(int)

    total_counts = pivot.sum(axis=0)
    sorted_columns = total_counts.sort_values(ascending=False).index
    pivot_sorted = pivot[sorted_columns]
    return pivot_sorted

In [ ]:
def create_line(data):
    data = data.to_crs('EPSG:4326')
    data = data['geometry']
    # Buat peta dengan lokasi awal berdasarkan rata-rata koordinat dari data linestring
    avg_lat = data.apply(lambda x: x.centroid.y).mean()
    avg_lon = data.apply(lambda x: x.centroid.x).mean()
    m = folium.Map(location=[avg_lat, avg_lon], zoom_start=15)

    # Tambahkan polyline untuk setiap linestring
    for linestring in data:
        coordinates = [(lat, lon) for lon, lat in linestring.coords]
        folium.PolyLine(locations=coordinates, color='purple', weight=8, opacity=0.7).add_to(m)

    # Tambahkan GeoJson dari data GeoPandas
    folium.GeoJson(data.to_json(), name='Garis Jalan').add_to(m)

    return m

In [ ]:
def create_plot_adjust(data):
    m = folium.Map(location=[-7.794847, 110.365608], zoom_start=25)

    # Add CircleMarkers for each point
    for index, row in data.iterrows():
        folium.CircleMarker(
            location=[row["adjusted_latitude"], row["adjusted_longitude"]],
            radius=5,  # Marker size
            color="blue",  # Marker color
            fill=True,
            fill_color="blue",  # Fill color of the marker
            fill_opacity=0.7,  # Opacity of the marker fill
            popup=f"User ID: {row['maid']}<br>Latitude: {row['adjusted_latitude']}<br>Longitude: {row['adjusted_longitude']}",
        ).add_to(m)
    
    return m

In [ ]:
gps.head()

In [ ]:
create_pivot(gps)

In [ ]:
gps['timestamp'] = pd.to_datetime(gps['timestamp'])
gps3['timestamp'] = pd.to_datetime(gps3['timestamp'])
gps4['timestamp'] = pd.to_datetime(gps4['timestamp'])

orang = gps[(gps['maid'] == '752aa534-b373-4660-9fb2-cc9f72b77be2' ) & (gps['timestamp'].dt.date == pd.to_datetime('2021-12-20').date())]
orang3 = gps3[(gps3['maid'] == '752aa534-b373-4660-9fb2-cc9f72b77be2' ) & (gps3['timestamp'].dt.date == pd.to_datetime('2021-12-20').date())]
orang4 = gps4[(gps4['maid'] == '752aa534-b373-4660-9fb2-cc9f72b77be2' ) & (gps4['timestamp'].dt.date == pd.to_datetime('2021-12-20').date())]

In [ ]:
orang3

In [ ]:
create_plot(orang)

In [ ]:
create_plot(orang3)

In [ ]:
create_plot(orang4)